# GPT2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 11.7 MB/s 
     |████████████████████████████████| 86 kB 4.5 MB/s 
     |████████████████████████████████| 6.6 MB 28.1 MB/s 
     |████████████████████████████████| 596 kB 48.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%cd /content/drive/MyDrive/CSNLP/Slang-Detection

/content/drive/MyDrive/CSNLP/Slang-Detection


In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import torch
import re
import pandas as pd
import pickle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

cuda


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
!pip install openpyxl

In [ ]:
from openpyxl import Workbook
from openpyxl  import load_workbook
import gc

In [ ]:
def event_generation(s,n,num,word):
  """
  s: string, input triggle samples
  n: int, generationt times
  num: int, used to find the generated sentence
  word: the word the sentence shoud include
  """

  input_ids = tokenizer.encode(s, return_tensors='tf')
  # print("s",s)
  # print("n",n)
  # print("num",num)
  # print("word",word)
  # print(input_ids.shape[1])


  tf.random.set_seed(10)

    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
  sample_outputs = model.generate(
      input_ids,
      do_sample=True, 
      max_length=input_ids.shape[1]+50, 
      top_k=50, 
      top_p=0.95, 
      temperature=0.5,
      num_return_sequences= 4*n
  )
  list2 = []
  for i, sample_output in enumerate(sample_outputs):
    # print(i)
    s = str(tokenizer.decode(sample_output, skip_special_tokens=True))
    # print(s)
    # print("pattern:",str(num+1)+"."+str(word)+":(.*?)\n")
    if(len(re.findall(str(num+1)+". "+str(word)+" : (.*?)\n", s)) >= 1):
        s2 = re.findall(str(num+1)+". "+str(word)+" : (.*?)\n", s)[0]
        if(len(re.findall("(.*?)"+str(word)+"(.*?)",s2)) >= 1):
          s2 = s2.replace('\xa0', '')
          list2.append(s2)
          if(len(list2)== n):
            return list2
  # print(len(list2))
  

In [ ]:
def example_gener(word,wordlist):
    s= ""
#     print("word",word)
#     print("wordlist",wordlist)
    for i in range(len(wordlist)):
#         print("i",i)
#         print("wordlist[i]",wordlist[i])
        s = s+str(i+1) + ". " + str(word) + " : "+str(wordlist[i])+"\n"
    s = s + str(len(wordlist)+1) + ". " +str(word) + " : "
    return s

In [ ]:
filedir = "data/data_cleaned_new.csv"
data_cleaned = pd.read_csv(filedir, index_col=0).sort_values(['word'])

temp_list = []
tempword = data_cleaned.iloc[0, 0]

trigger_list = []
trigger_word = []
trigger_len = []
for i in range(len(data_cleaned)):
  if (data_cleaned.iloc[i, 0] == tempword):
    temp_list.append(data_cleaned.iloc[i, 2])
  else:
    s = example_gener(tempword, temp_list)
    trigger_list.append(s)
    trigger_word.append(tempword)
    trigger_len.append(len(temp_list))
    tempword = data_cleaned.iloc[i, 0]
    temp_list = [data_cleaned.iloc[i, 2]]

In [ ]:
df_trigger = pd.DataFrame(columns = ['word', 'trigger', 'length'])
df_trigger['word'] = trigger_word
df_trigger['length'] = trigger_len
df_trigger['trigger'] = trigger_list
df_trigger = df_trigger.reset_index()
df_trigger.to_csv('data/trigger_data.csv')

In [ ]:
df_trigger = pd.read_csv('data/trigger_data.csv', index_col = 0)
failed = {'trigger': [], 'length': [], 'word': []}
succeed = {'trigger': [], 'length': [], 'word': [], 'generate':[]}

succeed_trigger = []
succeed_length = []
succeed_word = []
succeed_generate = []

failed_trigger = []
failed_length = []
failed_word = []

for i in range(len(df_trigger)):
  trigger = df_trigger.iloc[i, :]
  try:
    result = event_generation(trigger['trigger'], 10, trigger['length'], trigger['word'])
    # print(result)
    succeed_trigger.append(trigger['trigger'])
    succeed_length.append(trigger['length'])
    succeed_word.append(trigger['word'])
    succeed_generate.append(result)
    print('successfully generated {}'.format(i))
  except:
    failed_trigger.append(trigger['trigger'])
    failed_length.append(trigger['length'])
    failed_word.append(trigger['word'])
    print('failed generation {}'.format(i))
  if i%100 == 0:
    failed = dict()
    succeed = dict()
    failed['trigger'] = failed_trigger
    failed['length'] = failed_length
    failed['word'] = failed_word

    succeed['trigger'] = succeed_trigger
    succeed['length'] = succeed_length
    succeed['word'] = succeed_word
    succeed['generate'] = succeed_generate
    failed = pd.DataFrame(failed, columns = ['trigger', 'length', 'word'])
    succeed = pd.DataFrame(succeed, columns = ['trigger', 'length', 'word', 'generate'])

    failed.to_csv('data/failed.csv')
    succeed.to_csv('data/succeed.csv')

  # successfully generated 12304

successfully generated 0
successfully generated 1
successfully generated 2
successfully generated 3
successfully generated 4
successfully generated 5
successfully generated 6
successfully generated 7
successfully generated 8
successfully generated 9
successfully generated 10
successfully generated 11
successfully generated 12
successfully generated 13
successfully generated 14
successfully generated 15
successfully generated 16
successfully generated 17
successfully generated 18
successfully generated 19
successfully generated 20
successfully generated 21
successfully generated 22
successfully generated 23
successfully generated 24
successfully generated 25
successfully generated 26
successfully generated 27
successfully generated 28
successfully generated 29
successfully generated 30
successfully generated 31
successfully generated 32
successfully generated 33
successfully generated 34
successfully generated 35
successfully generated 36
successfully generated 37
successfully generated

Token indices sequence length is longer than the specified maximum sequence length for this model (1034 > 1024). Running this sequence through the model will result in indexing errors


successfully generated 147
failed generation 148
successfully generated 149
successfully generated 150
successfully generated 151
successfully generated 152
successfully generated 153
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "<ipython-input-4-c843dfa50f20>", line 27, in event_generation
    num_return_sequences= 4*n  File "/usr/local/lib/python3.7/dist-packages/transformers/generation_tf_utils.py", line 596, in generate
    forced_eos_token_id=forced_eos_token_id,  File "/usr/local/lib/python3.7/dist-packages/transformers/generation_tf_utils.py", line 1603, in _generate
    **model_kwargs,  File "/usr/local/lib/python3.7/dist-packages/transformers/generation_tf_utils.py", line 2351, in sample
    generated, finished_sequences, input_ids, cur_len, model_kwargs  File "/usr/local/lib/pytho

In [ ]:
fail_filename = open("data/failed.pkl","rb")
succ_filename = open("data/succeed.pkl","rb")
failed = pickle.load(fail_filename)
succeed = pickle.load(succ_filename)

In [ ]:
# debug
trigger = df_trigger.iloc[103, :]
s = trigger['trigger']
n = 10
num = trigger['length']
word = trigger['word']


input_ids = tokenizer.encode(s, return_tensors='tf')
# print("s",s)
# print("n",n)
# print("num",num)
# print("word",word)
# print(input_ids.shape[1])


tf.random.set_seed(10)

  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=input_ids.shape[1]+50, 
    top_k=50, 
    top_p=0.95, 
    temperature=0.5,
    num_return_sequences= 4*n
)
print('----')
list2 = []
for i, sample_output in enumerate(sample_outputs):
  print(i)
  s = str(tokenizer.decode(sample_output, skip_special_tokens=True))
  print(s)
  # print("pattern:",str(num+1)+"."+str(word)+":(.*?)\n")
  if(len(re.findall(str(num+1)+". "+str(word)+" : (.*?)\n", s)) >= 1):
      s2 = re.findall(str(word)+" : (.*?)\n", s)
      print(s2)
      if(len(re.findall("(.*?)"+str(word)+"(.*?)",s2)) >= 1):
        s2 = s2.replace('\xa0', '')
        list2.append(s2)
        if(len(list2)== n):
          print(list2)
# print(len(list2))


----
0
1. abide : "yeah, well. the dude abides."
2. abide :  "I'm gonna be honest, I'm not gonna lie to you. I'm not gonna lie to you. I'm not gonna lie to you. I'm not gonna lie to you. I'm not gonna lie to you. I
1
1. abide : "yeah, well. the dude abides."
2. abide :  "yeah, well. the dude abides."
3. abide :  "yeah, well. the dude abides."
4. abide :  "yeah, well. the dude abides."
5. abide
['"yeah, well. the dude abides."', '\xa0"yeah, well. the dude abides."', '\xa0"yeah, well. the dude abides."', '\xa0"yeah, well. the dude abides."']


TypeError: ignored

In [ ]:
fileaddress = "gdrive/MyDrive/CSNLP/data_cleaned.xlsx"
wb2 = load_workbook(filename = fileaddress)    # 打开文件, 默认可读写
sheet = wb2["data_cleaned"]
sheet2 = wb2["Sheet1"] #result

row = 2
row2 = 1
templist = []
tempword = sheet.cell(2,2).value #初始词

while(sheet.cell(row,2).value is not None):
    # print(row)
    if(sheet.cell(row,2).value == tempword):
        templist.append(sheet.cell(row,4).value)
        row = row + 1
    else:
        s = example_gener(tempword,templist)
        # print("event_generation")

        result = event_generation(s,10,len(templist),tempword)
        if(result is not None):
          for j in range(len(result)):
            sheet2.cell(row2,1).value = tempword
            sheet2.cell(row2,2).value = result[j]
            wb2.save(fileaddress)
            print("save success"+ str(row2))
            row2 = row2+1



        tempword = sheet.cell(row,2).value
        templist = []
        
    

print("done")
wb2.save(fileaddress)
gc.disable()

In [ ]:
# count max length
max([len(source) for source in trigger_list])

10530

In [ ]:
for s in trigger_list:
  input_ids = tokenizer.encode(s, return_tensors='tf')

Token indices sequence length is longer than the specified maximum sequence length for this model (1034 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
input_ids = []
attention_masks = []
for s in trigger_list[100:130]:
  encoded_dict = tokenizer(
        s,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors='pt',
        add_prefix_space = True
    )
  input_ids.append(encoded_dict['input_ids'])
  attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim = 0)
input_ids = input_ids.to(dtype = torch.int32)
print(input_ids.dtype)
attention_masks = torch.cat(attention_masks, dim = 0)
attention_masks = attention_masks.to(dtype = torch.int32)

torch.int32


In [ ]:
sample_outputs = model.generate(
    input_ids[:3, :],
    do_sample=True, 
    attention_mask = attention_masks[:3, :],
    max_length=288, 
    top_k=50, 
    top_p=0.95, 
    temperature=0.5,
    num_return_sequences= 4
)

In [ ]:
for sample_output in sample_outputs:
  s = str(tokenizer.decode(sample_output, skip_special_tokens=True))
  # print("___")
  # print(s)